In [ ]:
import numpy as np 
import pandas as pd 
import fastai
from fastai.tabular.all import *
from pandas_profiling import ProfileReport

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
sub = pd.read_csv("../input/tabular-playground-series-jul-2021/sample_submission.csv")

train['is_train']=True
test['is_train']= False
target = ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']

X = train.append(test)
X['date_time'] = pd.to_datetime(X['date_time'])

X['hr'] = X.date_time.dt.hour
cont_features = [x for x in X.columns if x not in target+['date_time','id','is_train']]
X['day'] =X.date_time.dt.weekday//5
X['satday'] = X.date_time.dt.weekday==5
X['hr1'] = X.date_time.dt.hour
X['hr2'] = X.date_time.dt.hour//5 ==0
X['lowh'] = X.absolute_humidity < 0.24
cat_features = ['day','satday','hr1','hr2','lowh']

X[target]= np.log(X[target]+1)
X[cont_features]= X[cont_features].astype(float)
print(cont_features)

In [ ]:
#ProfileReport(train)

In [ ]:
cbs = [ EarlyStoppingCallback(monitor='valid_loss', min_delta=0.0, patience=10),
        SaveModelCallback(monitor='valid_loss', comp=None, min_delta=0.0, 
                          fname='fastai', every_epoch=False, 
                          with_opt=False, reset_on_fit=True)] 

n=10
pred = sub[sub.columns[1:]]*0
log = pd.DataFrame(columns=['Loss Train','Loss Validation'])
train=X[X.is_train]
test= X[~X.is_train]
for m in target:
    p=pd.DataFrame([])
    for i in range(n):
        Xtrain = X[X.is_train]
        #Xtest = X[(~X.is_train) | (X.is_train & (X.fold == i))]
        #splits = IndexSplitter(Xtrain[Xtrain.fold ==i].index)(Xtrain)
        splits = RandomSplitter(valid_pct=0.2)(range_of(train))
        to = TabularPandas(
            train,
            y_names=m,
            y_block = RegressionBlock(),
            cont_names = cont_features,
            cat_names = cat_features,
            procs = [Categorify],
            splits=splits)

        loaders = to.dataloaders(bs=256//4)
        learn = tabular_learner(loaders,loss_func=nn.MSELoss(),cbs=cbs)
        learn.fit_one_cycle(10,0.01,wd=0.1)
        loss = learn.recorder.log[1:3]
        log.loc[len(log)] = loss
        dl = learn.dls.test_dl(test[cont_features+cat_features])
        p['Iter'+str(i)] = learn.get_preds(dl=dl)[0].numpy()[:,0]
        
    pred[m]= p.mean(axis=1).values
        
log.describe()

In [ ]:
sub[target] = np.exp(pred)-1
sub.to_csv("submission.csv", index=False)